# Using Wikipedia to Obtain Topic Key Words

This code downloads wikipedia pages, processes them and obtains the top 20 most frequent nouns and top 10 most frequent bigram words with at least one word being a noun.  This data is output into files that can later be used to tag subject topics.   

In [ ]:
import urllib.request as urlrequest
import wikipedia as wiki
import nltk
from bs4 import BeautifulSoup

#subjects and search terms
subjects = {'aid_effectiveness' : ['aid effectiveness'],
            'climate_change' : ['climate change'],
            'corruption_governance' : ['corruption','corruption and transparency','governance'],
            'data_technology' : ['data and technology'],
            'education' : ['education'],
            'finance_investment' : ['finance','investment'],
            'gender' : ['gender'],
            'global_health' : ['global health'],
            'infrastructure' : ['infrastructure'],
            #'interntl_inst' : ['international institutions'],
            'migration' : ['human migration'],
            'povertyineqgrowth' : ['poverty','inequality and growth'],
            'intltrade' : ['international trade']
}

def get_wikipedia_pages(subjects):

    #subjects is a dictionary pointing to list of search terms
    for key, val in subjects.items():
        text = ""
        for search_term in val:
            # 1. Grab  the list from wikipedia.
            wiki.set_lang('en')
            searches = wiki.search(search_term)
            print(wiki.search(search_term))
            #get the first two pages and concatenate or save these pages in folder
            for i in range(0,2):
                fname = re.sub('\s',"_",searches[i])
                s = wiki.page(searches[i])
                #fname = re.sub('\s',"_",searches[i])
                print(fname)
                html = urlrequest.urlopen(s.url).read()
                parser = BeautifulSoup(html, 'html.parser')
                text = text + parser.get_text()
  
        #Extract key words and replace with space
        text = re.sub('\W+', " ", text.lower())
        # Replace any non-letter, space, or digit character in the headlines.
        text = re.sub(r'[\d]',"",text)
        text = re.sub('\s+'," ",text)

        #place things into parts of speech and label as noun, verb etc.
        tokenized = nltk.tokenize.word_tokenize(text)
        pos_words = set(nltk.pos_tag(tokenized))
    
        #use the built in list of stop words 
        from nltk.corpus import stopwords
        stop = set(stopwords.words('english'))
        more_stop_words = ['articles','countries','country','effects','mediawiki','template','value','limit','information',
                          'january','october','wikipedia']
       
        #keep words that have length <= 4 (did this by lower length thresshold and checking for each the categories manually)
        keep_words = ['aid','aid','anti','bank','boy','boys','fund','gas','girl','homo','law','men','ipcc','oecd','poor','risk','role','sea','sex','sub']
        clean_split = [w for w in text.split() if w not in stop and w not in more_stop_words and (len(w) >= 5 or w in keep_words)]
        #print(set(nltk.pos_tag(clean_split)))        
        count_words = pd.Series(clean_split).value_counts()
        count_words = count_words[count_words >= 25]
        cnt_words = nltk.FreqDist(clean_split)
        
        NOUNS = ['NN', 'NNS', 'NNP', 'NNPS']
        
        most_freq_nouns = [w for w, cnt in cnt_words.most_common(30) if nltk.pos_tag([w])[0][1] in NOUNS]
        print(most_freq_nouns)
        
        #print(type(count_words))
        print(len(count_words))

        bigrams = list(nltk.bigrams(clean_split))
        cnt_bigrams = nltk.FreqDist(bigrams)
        most_freq_bigrams = [w for w, cnt in cnt_bigrams.most_common(10) if w[0] in most_freq_nouns or w[1] in most_freq_nouns and w[0] != w[1]]
        print(most_freq_bigrams)
        count_bigrams = pd.Series(bigrams).value_counts()
        count_bigrams = count_bigrams[count_bigrams >= 10]
        print(len(count_bigrams))
        
        fname = 'C:\\JobApplications\\Facebook\\TrainSubjects\\'+key+'.csv'
        with open(fname, 'wb') as f:
            pickle.dump(most_freq_nouns, f)
        
        fname = 'C:\\JobApplications\\Facebook\\TrainSubjects\\'+key+'_bigrams.csv'
        with open(fname, 'wb') as f:
            pickle.dump(most_freq_bigrams, f)
        
        #readout the subject files
        #most_freq_nouns.to_csv('C:\\JobApplications\\Facebook\\TrainSubjects\\'+key+'.csv')
        #most_freq_bigrams.to_csv('C:\\JobApplications\\Facebook\\TrainSubjects\\'+key+'_bigrams.csv')      
        
get_wikipedia_pages(subjects)